In [ ]:
# Check data availability
train_dir = data_dir / 'train'
val_dir = data_dir / 'validation'
test_dir = data_dir / 'test'

print("📊 Dataset Summary:")
print(f"Train directory: {train_dir} - Exists: {train_dir.exists()}")
print(f"Validation directory: {val_dir} - Exists: {val_dir.exists()}")
print(f"Test directory: {test_dir} - Exists: {test_dir.exists()}")

# Count images if directories exist
def count_images_in_dir(directory):
    """Count images in directory and subdirectories"""
    if not directory.exists():
        return 0
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len([f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    return count

if train_dir.exists():
    train_count = count_images_in_dir(train_dir)
    val_count = count_images_in_dir(val_dir)
    test_count = count_images_in_dir(test_dir)
    
    print(f"\n📈 Image Counts:")
    print(f"Training images: {train_count}")
    print(f"Validation images: {val_count}")
    print(f"Test images: {test_count}")
    print(f"Total: {train_count + val_count + test_count}")
else:
    print("\n⚠️  Dataset not found. Please download and organize the Cats vs Dogs dataset.")
    print("Expected structure:")
    print("data/")
    print("├── train/")
    print("│   ├── cats/")
    print("│   └── dogs/")
    print("├── validation/")
    print("│   ├── cats/")
    print("│   └── dogs/")
    print("└── test/")
    print("    ├── cats/")
    print("    └── dogs/")

In [ ]:
# Download Microsoft Cats vs Dogs dataset
import urllib.request
import tarfile

# Create data directory
data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

# Download dataset (note: this is a large file ~786MB)
print("⏳ Downloading Cats vs Dogs dataset...")
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

# Alternative: Use smaller sample or local files
# For demonstration, we'll assume data is already organized
print("📁 Data directory structure:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(str(data_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:3]:  # Show only first 3 files
        print(f'{subindent}{file}')
    if len(files) > 3:
        print(f'{subindent}... and {len(files) - 3} more files')

## 2️⃣ Load and Explore Dataset

### Download and Prepare Data
For this example, we'll use the Microsoft Cats vs Dogs dataset.

In [ ]:
# Import core libraries
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# Image processing
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

# Metrics and evaluation
from sklearn.metrics import (
    confusion_matrix, 
    classification_report,
    roc_curve,
    auc,
    roc_auc_score
)

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("✅ Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 1️⃣ Import Required Libraries

# 🖼️ Image Classifier com Deep Learning
## Classificação de Gatos vs Cães usando CNN

Este notebook implementa um classificador de imagens usando Redes Neurais Convolucionais (CNN) com TensorFlow/Keras.

### 📊 Objetivos
- ✅ Manipulação e preprocessamento de imagens
- ✅ Data Augmentation para melhorar robustez
- ✅ Construção de CNN personalizada
- ✅ Treinamento com monitoramento de overfitting
- ✅ Avaliação com métricas diversas
- ✅ Comparação com Transfer Learning